<a href="https://colab.research.google.com/github/TheLittleJimmy/CBB750/blob/main/EDA_minor_class_now_has_about_60_samples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
data_path='/content/drive/MyDrive/BIS550_CBB750/project/Code by message type.xlsx'

In [3]:
import pandas as pd

In [4]:
IG_P = pd.read_excel(data_path, sheet_name='Info Giving - Patient')
IG_C = pd.read_excel(data_path, sheet_name='Info Giving - Clinician')
IG_A = pd.read_excel(data_path, sheet_name='Info Giving - Auto')

IS_C = pd.read_excel(data_path, sheet_name='Info Seek - Clinician')
IS_P = pd.read_excel(data_path, sheet_name='Info Seek - Patient')

Emo_P = pd.read_excel(data_path, sheet_name='Emotion - Patient')
Emo_C = pd.read_excel(data_path, sheet_name='Emotion - Clinician')

P_C = pd.read_excel(data_path, sheet_name='Partnership - Clinician')
P_P = pd.read_excel(data_path, sheet_name='Partnership - Patient')

SDM_P = pd.read_excel(data_path, sheet_name='SDM - Patient')
SDM_C = pd.read_excel(data_path, sheet_name='SDM - Clinician',usecols=[0,1])

In [5]:
# Combine IG_P, IG_C, IG_A into one dataframe
IG = pd.concat([IG_P, IG_C, IG_A], axis=0, ignore_index=True)
# Combine IS_C, IS_P into one dataframe
IS = pd.concat([IS_C, IS_P], axis=0, ignore_index=True)
# Combine Emo_P, Emo_C into one dataframe
Emo = pd.concat([Emo_P, Emo_C], axis=0, ignore_index=True)
# Combine P_C, P_P into one dataframe
P = pd.concat([P_C, P_P], axis=0, ignore_index=True)
# Combine SDM_C, SDM_P into one dataframe
SDM = pd.concat([SDM_C, SDM_P], axis=0, ignore_index=True) # SDM - shared decision making # really small class, need data augmentation

In [6]:
Emo.shape

(32, 2)

In [7]:
# Add labels to each dataframe
IG['Label'] = 'Info Giving'
IS['Label'] = 'Info Seeking'
Emo['Label'] = 'Emotion'
P['Label'] = 'Partnership'
SDM['Label'] = 'SDM'
# Combine all dataframes into one dataframe
df = pd.concat([IG, IS, Emo, P, SDM], axis=0, ignore_index=True)
df

,Quotation Content,Codes,Label
0,197 I noticed that my medication list has not ...,Patient,Info Giving
1,Blood in urine and stool- began yesterday We a...,Patient,Info Giving
2,Can someone set up that appointment as I do no...,Patient,Info Giving
3,Dear Doctor Lacy I just was told my daughter w...,Patient,Info Giving
4,Dear Dr Chiang When I saw you last Thursday( M...,Patient,Info Giving
...,...,...,...
270,Hi That sounds reasonable I would do the colon...,Clinician,SDM
271,Hi Mr It is most likely a radiation side effec...,Clinician,SDM
272,Sounds good- I just put in prescription for- n...,Clinician,SDM
273,Dr I have not scheduled the colonoscopy as yet...,Patient,SDM


In [8]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Easy Data Augmentation (EDA)
Four operations:  
Synonym Replacement (SR): Randomly choose n words from the sentence that are not stop words. Replace each of these words with one of its synonyms chosen at random.  
Random Insertion (RI): Find a random synonym of a random word in the sentence that is not a stop word. Insert that synonym into a random position in the sentence. Do this n times.  
Random Swap (RS): Randomly choose two words in the sentence and swap their positions. Do this n times.  
Random Deletion (RD): Randomly remove each word in the sentence with probability p. 

For more details see github repo: https://github.com/jasonwei20/eda_nlp

default parameters:
num_aug (the number of augmented sentences to generate given a sentense) = 9  
alpha parameters (percent of words in the sentence that will be changed) = 0.1  
alpha parameters include alpha_sr, alpha_rd, alpha_ri, alpha_rs

In [9]:
# put messages labeled SDM to a file in the format label\tsentence
f=open("sdm.txt","w")
for msg in SDM["Quotation Content"]:
  f.write(f"0\t{msg.strip()}\n")
f.close()

In [10]:
# perform eda using default alpha parameters and num_aug=11, augment.py is downloaded from Wei & Zou's Github
%%python /content/drive/MyDrive/BIS550_CBB750/project/augment.py --input=sdm.txt --num_aug=11

generated augmented sentences with eda for sdm.txt to eda_sdm.txt with num_aug=11


/content/drive/MyDrive/BIS550_CBB750/project/eda.py:177: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not '']


In [11]:
# take a look at the output and save augmented sentences to a list
eda_sdm=[]
f=open("eda_sdm.txt","r")
for eda_msg in f:
  print(eda_msg.strip())
  eda_sdm.append(eda_msg.strip().split("\t")[1])
f.close()

0	hi that sounds reasonable i would do the colonoscopy more electively when no contract issues are noted best vik
0	hi that sounds reasonable i would do unspoilt the colonoscopy more electively when no pressing issues are noted best vik
0	hi that sounds reasonable i would do the colonoscopy more electively when no pressing issues cut are noted best vik
0	hi that sounds reasonable i would do the colonoscopy more electively when no pressing issues are noted best vik
0	hi that sounds reasonable i would do when colonoscopy more electively the no pressing issues are noted best vik
0	how do you do that sounds reasonable i would do the colonoscopy more electively when no pressing issues are noted best vik
0	hi that sounds reasonable i would do issues colonoscopy more electively when no pressing the are noted best vik
0	hi that sounds reasonable i would do the colonoscopy more electively when no pressing issues are hello noted best vik
0	hi that sounds reasonable i would do the colonoscopy mor

In [12]:
EDA_SDM=pd.DataFrame(data=zip(*[eda_sdm,["SDM" for i in range(len(eda_sdm))]]),columns=["Quotation Content","Label"])

In [13]:
Emo.shape

(32, 3)

In [14]:
# augment Partnership and Emotion as well

f=open("partner.txt","w")
for msg in P["Quotation Content"]:
  f.write(f"0\t{msg.strip()}\n")
f.close()

f=open("emotion.txt","w")
for msg in Emo["Quotation Content"]:
  f.write(f"0\t{msg.strip()}\n")
f.close()

In [16]:
%%python /content/drive/MyDrive/BIS550_CBB750/project/augment.py --input=partner.txt --num_aug=2

generated augmented sentences with eda for partner.txt to eda_partner.txt with num_aug=2


In [17]:
%%python /content/drive/MyDrive/BIS550_CBB750/project/augment.py --input=emotion.txt --num_aug=1

generated augmented sentences with eda for emotion.txt to eda_emotion.txt with num_aug=1


In [18]:
eda_p=[]
f=open("eda_partner.txt","r")
for eda_msg in f:
  #print(eda_msg.strip())
  eda_p.append(eda_msg.strip().split("\t")[1])
f.close()

eda_e=[]
f=open("eda_emotion.txt","r")
for eda_msg in f:
  #print(eda_msg.strip())
  eda_e.append(eda_msg.strip().split("\t")[1])
f.close()

In [19]:
EDA_P=pd.DataFrame(data=zip(*[eda_p,["Partnership" for i in range(len(eda_p))]]),columns=["Quotation Content","Label"])
EDA_Emo=pd.DataFrame(data=zip(*[eda_e,["Emotion" for i in range(len(eda_e))]]),columns=["Quotation Content","Label"])

In [20]:
EDA_SDM.shape[0],EDA_P.shape[0],EDA_Emo.shape[0]

(60, 54, 64)

In [21]:
augmented_df = pd.concat([IG.drop(columns=["Codes"]), IS.drop(columns=["Codes"]), EDA_Emo, EDA_P, EDA_SDM], axis=0, ignore_index=True)

In [22]:
augmented_df

,Quotation Content,Label
0,197 I noticed that my medication list has not ...,Info Giving
1,Blood in urine and stool- began yesterday We a...,Info Giving
2,Can someone set up that appointment as I do no...,Info Giving
3,Dear Doctor Lacy I just was told my daughter w...,Info Giving
4,Dear Dr Chiang When I saw you last Thursday( M...,Info Giving
...,...,...
393,gh and comprehensive metabolic panel blood tes...,SDM
394,gh and comprehensive metabolic panel blood tes...,SDM
395,gh and comprehensive metabolic panel tests res...,SDM
396,gh and comprehensive metabolous panel blood te...,SDM


In [23]:
augmented_df.to_csv("augmented_data.csv")

TODO:

Feature extraction

Wordnet vs UMLS

In [ ]:
IG["Quotation Content"][0]

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
#word_tokenize(IG["Quotation Content"][0])

In [ ]:
from nltk.tokenize import sent_tokenize
sent_tokenize(IG["Quotation Content"][0])

In [ ]:
from nltk.tokenize.punkt import PunktSentenceTokenizer
pst = PunktSentenceTokenizer()
pst.tokenize(IG["Quotation Content"][0])

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = tokenizer.tokenize(IG["Quotation Content"][0])

# Print segmented sentences
for sentence in sentences:
    print(sentence)

In [ ]:
%pip install transformers


In [ ]:
from transformers import AutoTokenizer

# Load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Define unpunctuated text
text = "HeisgoingtothemarkettobuygroceriesShewenttotheparkandplayedwithherfriends"

# Use tokenizer to segment sentences
sentences = tokenizer.tokenize(text, add_special_tokens=False)

# Merge adjacent tokens that belong to the same sentence
merged_sentences = []
current_sentence = ""
for token in sentences:
    current_sentence += token
    if token.endswith(".") or token.endswith("?") or token.endswith("!"):
        merged_sentences.append(current_sentence)
        current_sentence = ""

# Print segmented sentences
for sentence in merged_sentences:
    print(sentence)


In [ ]:
merged_sentences